In [ ]:
import polars as pl
import numpy as np

from sf_quant.data.exposures import load_exposures, get_exposures_columns
from sf_quant.data.factors import get_factor_names
from sf_quant.data.assets import load_assets, get_assets_columns

import sf_quant.data as sfd

import datetime as dt

from factor_singal_construction import construct_signal

/home/boobus/projects/factor_momentum/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-11 12:03:13,324	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
start = dt.date(2001,1,1)
end = dt.date(2025,1,1)

factors = [fac for fac in get_factor_names('style') if not fac in ['USSLOWL_MOMENTUM','USSLOWL_LTREVRSL']]
columns = ['date', 'barrid'] + factors

In [4]:

daily = load_assets(start=start, end=end, columns=['date', 'barrid', 'return', 'market_cap'], in_universe=True).join(
    load_exposures(start=start, end=end, columns=columns, in_universe=True),
    on=['barrid', 'date'],
    how='inner'
).sort(['barrid', 'date'])
daily

date,barrid,return,market_cap,USSLOWL_BETA,USSLOWL_DIVYILD,USSLOWL_EARNQLTY,USSLOWL_EARNYILD,USSLOWL_GROWTH,USSLOWL_LEVERAGE,USSLOWL_LIQUIDTY,USSLOWL_MGMTQLTY,USSLOWL_MIDCAP,USSLOWL_PROFIT,USSLOWL_PROSPECT,USSLOWL_RESVOL,USSLOWL_SIZE,USSLOWL_VALUE
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2013-07-31,"""USA06Z1""",-0.1595,6.006157e8,-2.291,-1.221,-2.28,-1.675,1.501,-0.863,-2.296,-1.798,-0.893,1.411,-0.461,2.771,-2.746,-1.229
2013-08-01,"""USA06Z1""",0.9585,6.0865392e8,-2.258,-1.231,-2.292,-1.67,1.497,-0.862,-2.295,-1.786,-0.884,1.411,-0.456,2.757,-2.741,-1.227
2013-08-02,"""USA06Z1""",-0.1582,6.0769086e8,-2.272,-1.229,-2.822,-1.672,1.497,-0.862,-2.241,-1.786,-0.886,1.412,-0.449,2.749,-2.742,-1.221
2013-08-05,"""USA06Z1""",2.2187,6.211737e8,-2.296,-1.228,-2.819,-1.668,1.493,-0.862,-2.222,-1.784,-0.85,1.396,-0.442,2.728,-2.728,-1.217
2013-08-06,"""USA06Z1""",-2.4806,6.0576474e8,-2.126,-1.23,-2.604,-1.67,1.496,-0.863,-2.246,-1.785,-0.883,1.4,-0.457,2.75,-2.74,-1.223
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-12-24,"""USBQOR1""",2.5872,3.5976e9,0.775,-0.693,0.078,0.064,0.043,-0.323,1.717,0.07,0.385,0.135,-0.107,-0.554,-2.285,-0.046
2024-12-26,"""USBQOR1""",4.293,3.7521e9,0.764,-0.693,0.081,0.053,0.041,-0.325,1.547,0.07,0.413,0.136,-0.109,-0.481,-2.26,-0.054
2024-12-27,"""USBQOR1""",-5.108,3.5604e9,0.776,-0.697,0.09,0.025,0.047,-0.332,1.387,0.07,0.384,0.138,-0.114,-0.444,-2.284,-0.073


In [5]:
monthly = (daily.with_columns(
    pl.col('date').dt.truncate('1mo').alias('month')
)
.group_by(['month', 'barrid']).agg(
    [((np.log(1 + pl.col('return')*.01).sum()
    .exp() - 1)*100)
    .alias('ret'),
    pl.col('market_cap').last()]
    +
    [pl.col({fac}).mean() for fac in factors]
)
.sort(['barrid', 'month'])
)


monthly

month,barrid,ret,market_cap,USSLOWL_BETA,USSLOWL_DIVYILD,USSLOWL_EARNQLTY,USSLOWL_EARNYILD,USSLOWL_GROWTH,USSLOWL_LEVERAGE,USSLOWL_LIQUIDTY,USSLOWL_MGMTQLTY,USSLOWL_MIDCAP,USSLOWL_PROFIT,USSLOWL_PROSPECT,USSLOWL_RESVOL,USSLOWL_SIZE,USSLOWL_VALUE
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2013-07-01,"""USA06Z1""",-0.1595,6.006157e8,-2.291,-1.221,-2.28,-1.675,1.501,-0.863,-2.296,-1.798,-0.893,1.411,-0.461,2.771,-2.746,-1.229
2013-08-01,"""USA06Z1""",-1.437697,5.9463375e8,-1.558273,-1.228727,-2.770909,-1.672682,1.481545,-0.857909,-2.185455,-1.777909,-0.938273,1.401318,-0.458727,2.767727,-2.757545,-1.223273
2013-09-01,"""USA06Z1""",-32.414789,4.0188375e8,-1.6811,-1.19185,-2.9308,-1.67185,1.46495,-0.84545,-2.17295,-1.761,-1.8357,1.41765,-0.06605,2.7664,-3.0269,-1.13445
2013-10-01,"""USA06Z1""",27.098245,5.174708e8,-0.706043,-1.203913,-2.918174,-1.665826,1.45713,-0.853174,-2.139609,-1.750739,-1.409522,1.424348,-0.002348,2.786261,-2.899652,-1.177478
2013-11-01,"""USA06Z1""",18.868171,6.071625e8,-0.22635,-1.19915,-2.90835,-1.51555,1.469,-0.85635,-2.05875,-1.77485,-1.1947,1.4449,-0.0129,2.8234,-2.83945,-1.1798
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-12-01,"""USBQND1""",-3.512,1.0524e8,-0.241,-0.709,-2.859,-2.812,-0.822,0.248,-0.735,2.531,-2.808,-2.664,-0.606,2.8,-4.563,-0.676
2024-12-01,"""USBQO11""",-1.4206,9.6186e8,-0.704,0.331,-2.966,-2.587,-0.57,-0.502,-0.047,-0.299,-1.24,-0.248,-0.328,1.335,-3.127,0.74
2024-12-01,"""USBQOE1""",-1.0209,4.5533e9,0.104,-0.501,-0.213,0.784,-1.118,0.782,-2.439,0.208,0.551,-0.249,-0.001,0.161,-2.119,1.577


In [13]:
#factor_signals = (pl.read_parquet('~/temp/factor_signals-z-score.parquet')
           #.drop('ret','lag_ret','rank','count')
           #.pivot(on='factor', index='month', values='signal')
           #)

factor_signals = construct_signal(start=start, end=end, type='rolling').pivot(on='factor', index='month', values='signal')

factor_signals

month,USSLOWL_BETA,USSLOWL_DIVYILD,USSLOWL_EARNQLTY,USSLOWL_EARNYILD,USSLOWL_GROWTH,USSLOWL_LEVERAGE,USSLOWL_LIQUIDTY,USSLOWL_MGMTQLTY,USSLOWL_MIDCAP,USSLOWL_PROFIT,USSLOWL_PROSPECT,USSLOWL_RESVOL,USSLOWL_SIZE,USSLOWL_VALUE
date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2002-01-01,-0.176405,0.015585,0.029436,0.115423,-0.034607,-0.006442,0.026421,-0.011052,-0.042436,0.011843,0.015086,-0.049562,-0.093513,0.025043
2002-02-01,-0.174559,0.034637,0.049503,0.106541,-0.018909,-0.014189,0.024207,-0.011946,-0.0234,0.011226,0.00789,-0.077019,-0.078652,0.006518
2002-03-01,-0.080574,0.041382,0.046469,0.090951,-0.00949,-0.029516,0.017102,-0.016663,-0.035283,0.009284,0.015165,-0.028104,-0.09755,0.003558
2002-04-01,-0.049517,0.052921,0.062701,0.071848,0.009347,-0.037652,0.019406,-0.007986,-0.042885,0.018797,0.011908,-0.029296,-0.095918,0.004247
2002-05-01,-0.140958,0.043059,0.060767,0.055243,-0.012117,-0.030158,0.023535,0.008788,-0.065328,0.028328,0.003129,-0.065571,-0.081103,0.02783
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-09-01,-0.016223,0.007206,-0.020154,0.017485,-0.00186,0.002655,0.001613,0.009695,0.001449,0.011676,0.003386,0.036598,0.022717,-0.004014
2024-10-01,0.0095,0.013172,-0.014371,0.004913,-0.006799,0.013225,-0.013458,0.008795,-0.002682,0.00561,0.002961,0.037343,0.003832,-0.007926
2024-11-01,0.026106,0.008672,-0.008411,-0.004432,-0.002943,0.019277,-0.021034,0.005953,0.009074,0.002183,0.011678,0.039124,0.000431,-0.010858


In [9]:
merged = monthly.join(factor_signals, on='month', how='left', suffix='_signal')
merged

month,barrid,ret,market_cap,USSLOWL_BETA,USSLOWL_DIVYILD,USSLOWL_EARNQLTY,USSLOWL_EARNYILD,USSLOWL_GROWTH,USSLOWL_LEVERAGE,USSLOWL_LIQUIDTY,USSLOWL_MGMTQLTY,USSLOWL_MIDCAP,USSLOWL_PROFIT,USSLOWL_PROSPECT,USSLOWL_RESVOL,USSLOWL_SIZE,USSLOWL_VALUE,USSLOWL_BETA_signal,USSLOWL_DIVYILD_signal,USSLOWL_EARNQLTY_signal,USSLOWL_EARNYILD_signal,USSLOWL_GROWTH_signal,USSLOWL_LEVERAGE_signal,USSLOWL_LIQUIDTY_signal,USSLOWL_MGMTQLTY_signal,USSLOWL_MIDCAP_signal,USSLOWL_PROFIT_signal,USSLOWL_PROSPECT_signal,USSLOWL_RESVOL_signal,USSLOWL_SIZE_signal,USSLOWL_VALUE_signal
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2013-07-01,"""USA06Z1""",-0.1595,6.006157e8,-2.291,-1.221,-2.28,-1.675,1.501,-0.863,-2.296,-1.798,-0.893,1.411,-0.461,2.771,-2.746,-1.229,0.022111,0.000567,-0.009281,0.019793,0.002189,0.033046,-0.003038,0.022337,0.027941,0.006703,0.005963,0.000633,0.003228,0.036727
2013-08-01,"""USA06Z1""",-1.437697,5.9463375e8,-1.558273,-1.228727,-2.770909,-1.672682,1.481545,-0.857909,-2.185455,-1.777909,-0.938273,1.401318,-0.458727,2.767727,-2.757545,-1.223273,0.042225,-0.007086,-0.008888,0.013752,0.005743,0.034347,0.001725,0.022954,0.026845,0.014675,0.002743,0.023664,-0.006157,0.043995
2013-09-01,"""USA06Z1""",-32.414789,4.0188375e8,-1.6811,-1.19185,-2.9308,-1.67185,1.46495,-0.84545,-2.17295,-1.761,-1.8357,1.41765,-0.06605,2.7664,-3.0269,-1.13445,0.020058,0.000345,0.006154,0.009542,0.00671,0.02854,-0.012873,0.016638,0.026099,0.006391,-0.000606,0.021367,-0.01692,0.037006
2013-10-01,"""USA06Z1""",27.098245,5.174708e8,-0.706043,-1.203913,-2.918174,-1.665826,1.45713,-0.853174,-2.139609,-1.750739,-1.409522,1.424348,-0.002348,2.786261,-2.899652,-1.177478,0.029665,0.000711,0.010787,0.01742,0.004293,0.023741,-0.010338,0.020363,0.037657,0.004171,0.00308,0.031828,-0.022141,0.027107
2013-11-01,"""USA06Z1""",18.868171,6.071625e8,-0.22635,-1.19915,-2.90835,-1.51555,1.469,-0.85635,-2.05875,-1.77485,-1.1947,1.4449,-0.0129,2.8234,-2.83945,-1.1798,0.029553,0.00093,0.010209,0.010456,0.01199,0.011312,-0.001672,0.009906,0.029748,0.007082,0.003159,0.03842,-0.024678,0.018274
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-12-01,"""USBQND1""",-3.512,1.0524e8,-0.241,-0.709,-2.859,-2.812,-0.822,0.248,-0.735,2.531,-2.808,-2.664,-0.606,2.8,-4.563,-0.676,0.022151,0.013879,-0.004133,0.003659,-0.012202,0.012614,-0.026681,0.008025,0.008224,-0.000277,0.010721,0.035855,0.000704,-0.019452
2024-12-01,"""USBQO11""",-1.4206,9.6186e8,-0.704,0.331,-2.966,-2.587,-0.57,-0.502,-0.047,-0.299,-1.24,-0.248,-0.328,1.335,-3.127,0.74,0.022151,0.013879,-0.004133,0.003659,-0.012202,0.012614,-0.026681,0.008025,0.008224,-0.000277,0.010721,0.035855,0.000704,-0.019452
2024-12-01,"""USBQOE1""",-1.0209,4.5533e9,0.104,-0.501,-0.213,0.784,-1.118,0.782,-2.439,0.208,0.551,-0.249,-0.001,0.161,-2.119,1.577,0.022151,0.013879,-0.004133,0.003659,-0.012202,0.012614,-0.026681,0.008025,0.008224,-0.000277,0.010721,0.035855,0.000704,-0.019452


In [10]:
prods = merged.with_columns(merged.columns)
for fac in factors:
    prods = prods.with_columns(
        (pl.col(fac) * pl.col(f'{fac}_signal')).alias(f'{fac}_signal_exposure')
    ).drop([fac, f'{fac}_signal'])

prods = prods.drop_nulls()
prods

month,barrid,ret,market_cap,USSLOWL_BETA_signal_exposure,USSLOWL_DIVYILD_signal_exposure,USSLOWL_EARNQLTY_signal_exposure,USSLOWL_EARNYILD_signal_exposure,USSLOWL_GROWTH_signal_exposure,USSLOWL_LEVERAGE_signal_exposure,USSLOWL_LIQUIDTY_signal_exposure,USSLOWL_MGMTQLTY_signal_exposure,USSLOWL_MIDCAP_signal_exposure,USSLOWL_PROFIT_signal_exposure,USSLOWL_PROSPECT_signal_exposure,USSLOWL_RESVOL_signal_exposure,USSLOWL_SIZE_signal_exposure,USSLOWL_VALUE_signal_exposure
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2013-07-01,"""USA06Z1""",-0.1595,6.006157e8,-0.050655,-0.000693,0.02116,-0.033153,0.003286,-0.028518,0.006975,-0.040163,-0.024951,0.009457,-0.002749,0.001753,-0.008864,-0.045137
2013-08-01,"""USA06Z1""",-1.437697,5.9463375e8,-0.065798,0.008707,0.024628,-0.023003,0.008508,-0.029467,-0.003769,-0.04081,-0.025188,0.020564,-0.001258,0.065494,0.016978,-0.053818
2013-09-01,"""USA06Z1""",-32.414789,4.0188375e8,-0.03372,-0.000411,-0.018036,-0.015952,0.00983,-0.024129,0.027973,-0.029299,-0.047909,0.00906,0.00004,0.05911,0.051214,-0.041981
2013-10-01,"""USA06Z1""",27.098245,5.174708e8,-0.020945,-0.000857,-0.031478,-0.029018,0.006255,-0.020255,0.022119,-0.035651,-0.053078,0.005941,-0.000007,0.088681,0.0642,-0.031918
2013-11-01,"""USA06Z1""",18.868171,6.071625e8,-0.006689,-0.001115,-0.029692,-0.015847,0.017613,-0.009687,0.003442,-0.017582,-0.03554,0.010232,-0.000041,0.108476,0.070071,-0.021559
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-12-01,"""USBQND1""",-3.512,1.0524e8,-0.005338,-0.00984,0.011816,-0.010289,0.01003,0.003128,0.019611,0.020312,-0.023092,0.000738,-0.006497,0.100395,-0.00321,0.013149
2024-12-01,"""USBQO11""",-1.4206,9.6186e8,-0.015594,0.004594,0.012258,-0.009465,0.006955,-0.006332,0.001254,-0.0024,-0.010197,0.000069,-0.003517,0.047867,-0.0022,-0.014394
2024-12-01,"""USBQOE1""",-1.0209,4.5533e9,0.002304,-0.006953,0.00088,0.002869,0.013642,0.009864,0.065075,0.001669,0.004531,0.000069,-0.000011,0.005773,-0.001491,-0.030675


In [11]:
signals = (prods.select(
    pl.col('month'),
    pl.col('barrid'),
    pl.col('ret'),
    pl.col('market_cap'),
    prods[:, 4:18].sum_horizontal().alias('signal')
)
)

signals

month,barrid,ret,market_cap,signal
date,str,f64,f64,f64
2013-07-01,"""USA06Z1""",-0.1595,6.006157e8,-0.192252
2013-08-01,"""USA06Z1""",-1.437697,5.9463375e8,-0.098232
2013-09-01,"""USA06Z1""",-32.414789,4.0188375e8,-0.054211
2013-10-01,"""USA06Z1""",27.098245,5.174708e8,-0.03601
2013-11-01,"""USA06Z1""",18.868171,6.071625e8,0.072081
…,…,…,…,…
2024-12-01,"""USBQND1""",-3.512,1.0524e8,0.120913
2024-12-01,"""USBQO11""",-1.4206,9.6186e8,0.008898
2024-12-01,"""USBQOE1""",-1.0209,4.5533e9,0.067545


In [12]:
signals.describe()

statistic,month,barrid,ret,market_cap,signal
str,str,str,f64,f64,f64
"""count""","""824417""","""824417""",824417.0,824417.0,824417.0
"""null_count""","""0""","""0""",0.0,0.0,0.0
"""mean""","""2013-06-24 13:32:00.672000""",null,0.849815,8.0573e9,0.008693
"""std""",null,null,14.523405,4.5572e10,0.150579
"""min""","""2002-01-01""","""USA06Z1""",-99.835954,573620.0,-1.1037
"""25%""","""2007-10-01""",null,-5.605095,4.055115e8,-0.057847
"""50%""","""2013-07-01""",null,0.618154,1.1700e9,0.006777
"""75%""","""2019-04-01""",null,6.769859,3.9071e9,0.069659
"""max""","""2024-12-01""","""USBQOR1""",1625.051659,3.7853e12,1.270495


In [29]:
signals.write_parquet('~/temp/asset_signals-lookback.parquet')

In [40]:
signals.write_parquet('~/temp/asset_signals-z-score.parquet')

In [ ]:
signals.write_parquet('~/temp/asset_signals-ls.parquet')